# Model Setup and Embedding Pipeline

This notebook loads the EmbeddingGemma model, sets up the embedding pipeline, and demonstrates how to compute embeddings for text.


In [1]:
# Import functions from the scripts directory
from src.models.embedding_pipeline import load_embeddinggemma_model, embed_texts, compute_cosine_similarity
import torch


/home/goble54/spark-dev-workspace/Lora-for-embedding-models/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load EmbeddingGemma Model

The EmbeddingGemma model is a 300M parameter multilingual embedding model that produces 768-dimensional embeddings.


In [2]:
# Load the model and tokenizer
# Note: Make sure you've accepted the license and are logged into Hugging Face
tokenizer, model = load_embeddinggemma_model()

# Check device
device = next(model.parameters()).device
print(f"Model loaded on device: {device}")
print(f"Model type: {type(model).__name__}")


/home/goble54/spark-dev-workspace/Lora-for-embedding-models/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(


Model loaded on device: cuda:0
Model type: Gemma3TextModel


## Test Embedding Computation

Let's compute embeddings for a few example sentences and verify the pipeline works correctly.


In [3]:
# Example sentences to test
s1 = "The cat sits on the mat."
s2 = "A cat is sitting on a rug."
s3 = "The weather is sunny today."

# Compute embeddings using the embedding pipeline
embeddings = embed_texts([s1, s2, s3], model, tokenizer)

print(f"Embedding shape: {embeddings.shape}")
print(f"Expected shape: (3, 768)")
print(f"Embeddings are normalized: {torch.allclose(torch.norm(embeddings, dim=1), torch.ones(3), atol=1e-5)}")


Embedding shape: torch.Size([3, 768])
Expected shape: (3, 768)
Embeddings are normalized: True


## Compute Cosine Similarities

Since embeddings are normalized, cosine similarity equals the dot product. Let's see how similar our example sentences are.


In [4]:
# Compute cosine similarity matrix
cos_sim_matrix = compute_cosine_similarity(embeddings)

# Display the similarity matrix
import pandas as pd
import numpy as np

sentences = [s1, s2, s3]
sim_df = pd.DataFrame(
    cos_sim_matrix.numpy(),
    index=[f"S{i+1}" for i in range(len(sentences))],
    columns=[f"S{i+1}" for i in range(len(sentences))]
)

print("Cosine Similarity Matrix:")
print(sim_df.round(3))

print("\nSentence pairs:")
for i, s in enumerate(sentences, 1):
    print(f"S{i}: '{s}'")


Cosine Similarity Matrix:
       S1     S2     S3
S1  1.000  0.892  0.614
S2  0.892  1.000  0.595
S3  0.614  0.595  1.000

Sentence pairs:
S1: 'The cat sits on the mat.'
S2: 'A cat is sitting on a rug.'
S3: 'The weather is sunny today.'


## Expected Behavior

- S1 and S2 (both about cats on floor) should have high similarity (~0.8-0.9)
- S1 and S3 (cat vs weather) should have low similarity (~0.1-0.3)
- S2 and S3 should also have low similarity

The diagonal should be 1.0 (each sentence compared to itself).
